In [1]:
import os, sys
currentFolder = os.path.abspath('')
try:
    sys.path.remove(str(currentFolder))
except ValueError: # Already removed
    pass

projectFolder = 'C:/Users/abjawad/Documents/GitHub/local-attention-model'
sys.path.append(str(projectFolder))
os.chdir(projectFolder)
print( f"current working dir{os.getcwd()}")

current working dirC:\Users\abjawad\Documents\GitHub\local-attention-model


In [2]:
import torch
from  torch.utils.data import DataLoader
import torch.nn as nn
from models.builder import EncoderDecoder as segmodel
from dataloader.cfg_defaults import get_cfg_defaults
from utils.lr_policy import WarmUpPolyLR
from utils.init_func import init_weight, group_weight
from config_cityscapes import *
import yaml
import os
from dataloader.cityscapes_dataloader import CityscapesDataset
from val_segformer_rgbonly import val_cityscape
import matplotlib.pyplot as plt
import numpy as np

c:\Users\abjawad\.conda\envs\pt_cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\abjawad\.conda\envs\pt_cuda\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [3]:
config_path = 'C:/Users/abjawad/Documents/GitHub/local-attention-model/dataloader/cityscapes_rgbd_config.yaml'
# with open(config_path) as info:
#     info_dict = yaml.load(info, Loader=yaml.FullLoader)
cfg = get_cfg_defaults()
cfg.merge_from_file(config_path)
cfg.freeze()

data_mean = [0.291, 0.329, 0.291]
data_std = [0.190, 0.190, 0.185]

In [4]:
cityscapes_test = CityscapesDataset(cfg, split='val')
test_loader = DataLoader(cityscapes_test, batch_size=1, shuffle=False, num_workers=4) 
print(f'total test sample: {len(cityscapes_test)} v_iteration:{len(test_loader)}')



Using RGB input
Using RGB input
Found 500 val images
total test sample: 500 v_iteration:500


In [5]:
for i, data in enumerate(test_loader):
    img = data['image']
    # Split the [ 1, 3, 1024, 2048] image into two [3, 1024, 1024] images
    # img1 = img[:, :, 0:1024]
    # img2 = img[:, :, 1024:2048]


    img1, img2 = np.split(img.squeeze(), 2, axis=2)

    out = torch.cat((img1, img2), dim=2)
    
    print(out.shape)

    # add a dimentation to the image
    out = out.unsqueeze(0)

    print(out.shape)

    # print(img1.shape, img2.shape, out.shape)

    # # Create a figure for plotting
    # fig, axes = plt.subplots(1, 2)

    # # Plot the first image
    # axes[0].imshow(np.transpose(img1, (1, 2, 0)))  # Transpose the axes for correct display
    # axes[0].set_title('Image 1')
    # axes[0].axis('off')

    # # Plot the second image
    # axes[1].imshow(np.transpose(img2, (1, 2, 0)))  # Transpose the axes for correct display
    # axes[1].set_title('Image 2')
    # axes[1].axis('off')

    # plt.show()

    break


torch.Size([3, 1024, 2048])
torch.Size([1, 3, 1024, 2048])


In [6]:
criterion = nn.CrossEntropyLoss(reduction='mean', ignore_index=config.background)
model = segmodel(cfg=config, criterion=criterion, norm_layer=nn.BatchNorm2d, test=True)
model = nn.DataParallel(model, device_ids = config.device_ids)
# model.to(f'cuda:{model.device_ids[0]}', non_blocking=True)

# <----------------- load model ----------------->
# saved_model_path = os.path.join(config.checkpoint_dir, "09-12-23_1501", 'model_400.pth')
# print(saved_model_path)
# # exit()
state_dict = model.state_dict()
# model.load_state_dict(state_dict['model'], strict=True)
print(f'model loaded')
# epoch = state_dict['epoch']

# # model.eval()


15 16:49:55 Using backbone: Segformer-B2
15 16:49:56 Using MLP Decoder


model loaded


In [7]:
# val_loss, val_mean_iou = val_cityscape(0, test_loader, model)
val_cityscape(0, test_loader, model)

imgs:torch.Size([1, 3, 1024, 2048]) gts:torch.Size([1, 1024, 2048])
img1:torch.Size([1, 3, 1024, 1024]) img2:torch.Size([1, 3, 1024, 1024])
gts1:torch.Size([1, 1024, 1024]) gts2:torch.Size([1, 1024, 1024])
out shape:  torch.Size([1, 19, 1024, 2048]) torch.Size([1, 19, 1024, 1024]) torch.Size([1, 19, 1024, 1024])


RuntimeError: Encountered different devices in metric calculation (see stacktrace for details). This could be due to the metric class not being on the same device as input. Instead of `metric=MulticlassJaccardIndex(...)` try to do `metric=MulticlassJaccardIndex(...).to(device)` where device corresponds to the device of the input.